# 佐賀県の人口推移

In [ ]:
import pandas
import matplotlib.pyplot as plt
import matplotlib.axes as axes
import japanize_matplotlib

In [ ]:
#url = 'http://data.bodik.jp/dataset/77e0cc66-c15d-4473-b3df-2664fe8e2e63/resource/8dc71515-526a-4168-866c-05d2cc8dad7b/download/jinkou.xlsx'
url = 'jinkou.xlsx'

In [ ]:
#エクセル中の和暦ラベルと西暦の対応付け
# wareki ={'大正9年':1920, '昭和5年':1930, '10年':1935, '15年':1940,
#         '20年':1945, '25年':1950, '30年':1955, '35年':1960,'40年':1965,
#         '45年':1970, '50年':1975, '55年':1980, '60年':1985, '平成2年':1990,
#         '7年':1995, '12年':2000, '17年':2005, '22年':2010, '27年':2015}
def convertWareki(columns:pandas.Index) -> dict[str,int]:
    y = 1930
    wareki:dict[str,int] = dict()
    for k in columns:
        if k == '大正9年':
            wareki[k]=1920
        else:
            wareki[k]=y
            y+=5
    return wareki

In [ ]:
def getIndex(url:str)->dict[str,str]:
    """
    URLで指定されたExcelで、V列に対応する適切な行名を取得する
    """
    dd = pandas.read_excel(url, header = 3,  usecols = 'A:B,V',
        index_col = 2, skiprows = [4, 18, 27], skipfooter = 8) 
    newIndex:dict[str,str] = dict()
    for k in dd.index:
        v = dd['項'][k]
        if type(v) != str:
            v = dd['目'][k]
        newIndex[k] = v.strip()
    return newIndex

In [ ]:
def readData(url:str)->pandas.DataFrame:
    """
    urlで指定されたExcelファイルを読む
    """
    data = pandas.read_excel(url, header = 3, usecols = 'C:V',
             index_col = 19, skiprows = [4, 18, 27], skipfooter = 8) 
    keys: dict[str, str] = getIndex(url)
    wareki = convertWareki(data.columns)
    return data.rename(columns = wareki, index = keys)

In [ ]:
def plotSub(yearList:list[int], data:pandas.DataFrame,ax:axes.Axes):
    """
    yearListで指定された年の棒グラフを描画
    """
    #必要なデータに成形する：対象となる年のデータを取得し、1/1000にする
    year_range=['0～14歳','15～64歳','65歳以上']

    df=data.filter(yearList)#対象となる年の列だけを抽出
    #対象となる列だけを抽出して、1/1000
    df2=df.filter(year_range,axis='index')/1000
    df3=df2.transpose()#行と列を入れ替える

    color = ['g', 'y', 'r']
    label = ['年少', '生産年齢', '老年']

    df3.plot(kind='bar',stacked=True,ax=ax,color=color,label=label)


メイン

In [ ]:
#Excelの内容をDataFrameへ
data = readData(url)

fig, ax =plt.subplots(facecolor = 'white')
ax.set_title('佐賀県の人口 (国勢調査から)') #図タイトル
ax.set_xlabel('年')
ax.set_ylabel('人口 (千人)')
ax.set_ylim(0, 1200)

yearList = list()
year = 1945
while year < 2020:
    yearList.append(year)
    year += 10
plotSub(yearList, data, ax)

fig.savefig('SagaPopulationStackedBar.pdf') #画像保存
plt.show()